In [1]:
# Import dependencies
import os
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import pandas as pd
import numpy as np
import tensorflow as tf
import logging
from tqdm import tqdm

# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, NUM_CLASSES
from src.model.tensorflow_utils import load_and_split_dataset, apply_augmentation_to_dataset, oversample_minority_classes, optimize_dataset, count_dataset_size
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_loss_monitor, setup_garbage_collector, get_metrics
from src.model.tensorflow_utils import calculate_class_weights, show_class_weights, start_or_resume_training, analyze_class_distribution
from src.model.densnet.tensorflow_densnet_basic import build_densenet

# Input Data
DATASET_SIZE = 102697
initial_epoch = 21
resume_training = True
checkpoint_path = '/Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DensNet_v4/checkpoints/cp-0020.keras'
model_name = "Simplified_DensNet_v4"
tfrecord_path = f"{TF_RECORD_DATASET}/chest_xray_data.tfrecord"

In [2]:
# Load Dataset
batch_size = 32
shuffle_buffer_size = 1000
tfrecord_buffer_size = TF_BUFFER_SIZE
dataset_size = DATASET_SIZE

train_ds, val_ds, test_ds = load_and_split_dataset(tfrecord_path, shuffle_buffer_size, tfrecord_buffer_size, dataset_size)

I0000 00:00:1741496613.325727 30394108 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1741496613.325770 30394108 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Optimize Dataset for rare clasess
class_weights = calculate_class_weights(train_ds, NUM_CLASSES)
train_ds = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, NUM_CLASSES)

In [4]:
steps_per_epoch = int(count_dataset_size(train_ds, None) / 32)
validation_steps = int(count_dataset_size(val_ds, None) / 32)

# Testing
train_ds = apply_augmentation_to_dataset(train_ds)
train_ds = optimize_dataset(train_ds, batch_size)
val_ds = optimize_dataset(val_ds, batch_size)
test_ds = optimize_dataset(test_ds, batch_size)

Counting samples: 108110 samples [01:08, 1580.48 samples/s] 
Counting samples: 15404 samples [01:23, 184.71 samples/s] 


In [5]:
# Setup Model Deps
# Setup Loggers
logger = setup_logger()
training_logger = setup_training_logger(logger, batch_size, 100)
metrics_monitor = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
loss_monitor = setup_loss_monitor(MODELS_PATH, model_name, logger, val_ds, resume_training=resume_training, initial_epoch=initial_epoch)
garbage_collector = setup_garbage_collector(logger)
metrics = get_metrics()

# Setup compile arguments

# Model V1
# binary_loss_v1 = BinaryCrossentropy(label_smoothing=0.01, from_logits=False)
# Model V2
# focal_loss_v2  = BinaryFocalCrossentropy(gamma=2.0, alpha=0.25, from_logits=False, label_smoothing=0.01)
# Model V3
# focal_loss_v3  = BinaryFocalCrossentropy(gamma=2.0, alpha=0.25, from_logits=False, label_smoothing=0.01)
# Model V4
focal_loss_v4  = BinaryFocalCrossentropy(gamma=2.5, alpha=0.3, from_logits=False, label_smoothing=0.01)
reduce_lr = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)

epoch_mode = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

2025-03-09 06:08:21 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DensNet_v4/train_metrics.csv
2025-03-09 06:08:21 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DensNet_v4/val_metrics.csv
2025-03-09 06:08:21 - INFO - Cleaned training metrics file, kept 67560 records before epoch 21
2025-03-09 06:08:21 - INFO - Cleaned validation metrics file, kept 20 records before epoch 21
2025-03-09 06:08:21 - INFO - Found 67560 existing training records
2025-03-09 06:08:21 - INFO - Found 20 existing validation records
2025-03-09 06:08:21 - INFO - Cleaned loss analysis metrics file, kept 315 records before epoch 21
2025-03-09 06:08:21 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DensNet_v4/loss_analysis_metrics.csv


In [6]:
# Model Training
model = build_densenet(NUM_CLASSES)
compile_kwargs = {'optimizer': Adam(learning_rate=0.0001), 'loss': focal_loss_v4, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    30,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, loss_monitor, garbage_collector], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-03-09 06:08:22 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DensNet_v4/train_metrics.csv
2025-03-09 06:08:22 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DensNet_v4/val_metrics.csv
2025-03-09 06:08:22 - INFO - Cleaned training metrics file, kept 67560 records before epoch 21
2025-03-09 06:08:22 - INFO - Cleaned validation metrics file, kept 20 records before epoch 21
2025-03-09 06:08:22 - INFO - Found 67560 existing training records
2025-03-09 06:08:22 - INFO - Found 20 existing validation records
2025-03-09 06:08:22 - INFO - Cleaned loss analysis metrics file, kept 315 records before epoch 21
2025-03-09 06:08:22 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DensNet_v4/loss_analysis_metrics.csv


Loading full model from checkpoint: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DensNet_v4/checkpoints/cp-0020.keras


2025-03-09 06:08:23 - INFO - 
=== Training Started ===

2025-03-09 06:08:23 - INFO - Batch Size: 32
2025-03-09 06:08:23 - INFO - Optimizer: Adam
2025-03-09 06:08:23 - INFO - 

2025-03-09 06:08:23 - INFO - 
=== Starting Epoch 21 ===



Epoch 21/30
3378/3378 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step - accuracy: 0.9066 - auc: 0.6621 - f1_score: 0.0131 - loss: 0.0397 - precision: 0.2541 - recall: 0.0087   

2025-03-09 06:35:18 - INFO - 
=== Epoch 21 Summary ===
2025-03-09 06:35:18 - INFO - Time: 1614.46s
2025-03-09 06:35:18 - INFO - Training   - accuracy: 0.9062 - auc: 0.6686 - f1_score: 0.0148 - loss: 0.0401 - precision: 0.2475 - recall: 0.0101 - learning_rate: 0.0001
2025-03-09 06:35:18 - INFO - Validation - accuracy: 0.9138 - auc: 0.6798 - f1_score: 0.0013 - loss: 0.0544 - precision: 0.1379 - recall: 0.0008
2025-03-09 06:35:18 - INFO - ==================================================



...

2025-03-09 12:34:33 - INFO - 
=== Epoch 30 Summary ===
2025-03-09 12:34:33 - INFO - Time: 1575.84s
2025-03-09 12:34:33 - INFO - Training   - accuracy: 0.9067 - auc: 0.7044 - f1_score: 0.0346 - loss: 0.0373 - precision: 0.3747 - recall: 0.0241 - learning_rate: 0.0001
2025-03-09 12:34:33 - INFO - Validation - accuracy: 0.9124 - auc: 0.6995 - f1_score: 0.0077 - loss: 0.0549 - precision: 0.1914 - recall: 0.0063
2025-03-09 12:34:33 - INFO - ==================================================



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10

2025-03-09 12:38:07 - INFO - 
Loss Analysis - Atelectasis
2025-03-09 12:38:07 - INFO - Confidence Distribution:
2025-03-09 12:38:07 - INFO - -- High (>0.9): 0.00%
2025-03-09 12:38:07 - INFO - -- Medium (0.6-0.9): 0.00%
2025-03-09 12:38:07 - INFO - -- Uncertain (0.4-0.6): 2.80%
2025-03-09 12:38:07 - INFO - -- Low (<0.4): 97.20%
2025-03-09 12:38:07 - INFO - Performance:
2025-03-09 12:38:07 - INFO - -- True Positives: 0
2025-03-09 12:38:07 - INFO - -- False Positives: 0
2025-03-09 12:38:07 - INFO - -- Loss Contribution: 0.0260
2025-03-09 12:38:07 - INFO - Average Confidence:
2025-03-09 12:38:07 - INFO - -- Correct Predictions: 29.25%
2025-03-09 12:38:07 - INFO - -- Incorrect Predictions: 33.05%
2025-03-09 12:38:07 - INFO - 
Loss Analysis - Cardiomegaly
2025-03-09 12:38:07 - INFO - Confidence Distribution:
2025-03-09 12:38:07 - INFO - -- High (>0.9): 0.00%
2025-03-09 12:38:07 - INFO - -- Medium (0.6-0.9): 0.63%
2025-03-09 12:38:07 - INFO - -- Uncertain (0.4-0.6): 6.56%
2025-03-09 12:38:07 

3378/3378 ━━━━━━━━━━━━━━━━━━━━ 1793s 524ms/step - accuracy: 0.9068 - auc: 0.7002 - f1_score: 0.0343 - loss: 0.0371 - precision: 0.3834 - recall: 0.0236 - val_accuracy: 0.9124 - val_auc: 0.6995 - val_f1_score: 0.0077 - val_loss: 0.0549 - val_precision: 0.1914 - val_recall: 0.0063 - learning_rate: 1.0000e-04


2025-03-09 12:38:10 - INFO - 
=== Training Completed! ===

2025-03-09 12:38:10 - INFO - Final Metrics: accuracy: 0.9067 - auc: 0.7044 - f1_score: 0.0346 - loss: 0.0373 - precision: 0.3747 - recall: 0.0241 - val_accuracy: 0.9124 - val_auc: 0.6995 - val_f1_score: 0.0077 - val_loss: 0.0549 - val_precision: 0.1914 - val_recall: 0.0063



In [7]:
# Initialize the evaluator
from src.model.tensorflow_model_evaluation import ModelEvaluation

mappings_path = f"{TF_RECORD_DATASET}/label_mappings.csv"
evaluator = ModelEvaluation(
    model=model, 
    model_name=model_name, 
    test_dataset=test_ds,
    label_mappings_path=mappings_path,
    output_dir=MODELS_PATH
)

# Cell 2: Generate predictions and basic metrics
metrics = evaluator.evaluate()


Generating predictions...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1

In [8]:
evaluator.generate_classification_report()


Classification Report:
                    precision    recall  f1-score   support

       Atelectasis       0.00      0.00      0.00      1712
      Cardiomegaly       0.26      0.22      0.24       416
     Consolidation       0.00      0.00      0.00       669
             Edema       0.23      0.01      0.02       341
          Effusion       0.62      0.01      0.02      1968
         Emphysema       0.00      0.00      0.00       379
          Fibrosis       0.00      0.00      0.00       272
            Hernia       0.02      0.12      0.03        33
      Infiltration       0.33      0.00      0.00      2923
              Mass       0.00      0.00      0.00       857
        No Finding       0.00      0.00      0.00      7763
            Nodule       0.00      0.00      0.00       921
Pleural_Thickening       0.00      0.00      0.00       504
         Pneumonia       0.00      0.00      0.00       231
      Pneumothorax       0.00      0.00      0.00       738

         micro

In [9]:
evaluator.plot_confusion_matrices()


Generating combined confusion matrix...
Saved combined confusion matrix to: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DensNet_v4/combined_confusion_matrix.png


In [10]:
for image, labels in test_ds.take(1):
    if len(image.shape) == 4:
        image = image[0]
        labels = labels[0]
    
    evaluator.visualize_prediction(image=image, true_labels=labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 664ms/step
